In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
# pip install pypdf

In [6]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [7]:
from typing import Any, Dict, List

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.task_publisher import TaskPublisher

In [8]:
_sqs = None
_processor = None
_queue_url = None

In [9]:
with open("../config/tenants/dev.yaml", "r", encoding="utf-8") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {}) or {}
dynamodb = boto3.resource("dynamodb", region_name=dynamodb_config.get("region"))

messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))
contacts_table = dynamodb.Table(dynamodb_config.get("contacts_table"))
tasks_table = dynamodb.Table(dynamodb_config.get("tasks_table"))

sqs_config = tenant_config.get("sqs", {}) or {}
_sqs = boto3.client("sqs", region_name=sqs_config.get("region"))

_queue_url = (sqs_config.get("tasks_url"))

s3_config = tenant_config.get("s3", {}) or {}
_s3 = boto3.client("s3", region_name=s3_config.get("region"))

_processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    contacts_table=contacts_table,
    tasks_table=tasks_table,
    s3_client=_s3,
    task_publisher=TaskPublisher(_sqs, _queue_url),
)

print("INIT: processor ready")

INIT: processor ready


In [10]:
import json

body = {
 "task_id": "916b9e8d-17da-4d62-8b67-3969a196e0f5",
 "event_key": "PROCESSED",
 "agent_type": "ACCOUNTING_ASSISTANT",
 "context_key": {
  "identity": "whatsapp:525571969848"
 },
 "payload": {},
 "process_type": "WHATSAPP_CONVERSATION",
 "task_type": "REPROCESS_PENDING_FILES",
 "timestamp_epoch": 1766379648,
 "timestamp_iso": "2025-12-22T05:00:48.738757+00:00"
}
task = json.dumps(body)

In [11]:
try:
    message_id = "Fake"
    processed, remaining = _processor.process(task)
    print(f"PROCESS: messageId={message_id} processed={processed} remaining={remaining}")

    if remaining < 0:
        print(f"DROP: messageId={message_id} stale")

    delay = max(0, min(int(remaining), 43200))
    print(f"DEFER: messageId={message_id} delay={delay}s")

except Exception as e:
    print(f"ERROR: messageId={message_id} err={repr(e)}")

1111
{'process_type': 'WHATSAPP_DOCUMENT_PIPELINE', 'business_key': 'wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDJBOERCRDg0MDdGRjQwMThDQzVCAA==#729646526857388', 'state': 'WAITING_PASSWORD'}
PROCESS: messageId=Fake processed=True remaining=0
DEFER: messageId=Fake delay=0s
